In [1]:
import pandas as pd
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.multioutput import MultiOutputRegressor

### Helper Functions

In [2]:
def loadData(file):
    data = pd.read_csv(file)
    print('Raw shape: ',data.shape)
    data['Date'] = pd.to_datetime(data.Date)
    print('Days: ',len(set(data.Date)))
    return data

In [3]:
def getTimeSeries(df):
    table = pd.pivot_table(df, values='vehicle_count', index=['Date','Hour'],
                    columns=['DOLocationID'], aggfunc=np.sum, fill_value=0)
    return table

In [4]:
def zscoreNormalizeSpatial(matrix):
    m = matrix.copy()
    for i in range(m.shape[0]):
        m[i, :] = (m[i, :] - m[i, :].mean()) / (m[i, :].std()+1e-10)
        
    return m

In [5]:
def standardize(matrix):
    m = matrix.copy()
    scaler = StandardScaler()
    scaler.fit(m)
    t = scaler.transform(m)
    return scaler, t

In [6]:
def inverse_standardize(matrix, scaler):
    t = matrix.copy()
    return scaler.inverse_transform(t)

In [7]:
def addLag(dataset, maxlag, lagColumns):
    dataset_list = [dataset]

    for l in range(1, maxlag+1):
        df = dataset.shift(l)
        df = df[lagColumns]
        df.columns = [c+'_lag_'+str(l) for c in df.columns]
        dataset_list.append(df)

    dataset = pd.concat(dataset_list, axis=1).dropna()
    return dataset

In [8]:
def get_rmse(matrix1, matrix2):
    sumSquareError = np.mean(np.power(matrix1 - matrix2,2))
    rmse = np.power(sumSquareError,0.5)
    return rmse

#### Preparing Data

In [9]:
hub = 'Penn'
tune_hyp_params = False

In [10]:
dataDir = '/home/urwa/Documents/Projects/NYU Remote/project/data/processedData/'
file = dataDir + hub + 'VehiceByHour.csv'

In [11]:
data = loadData(file)

Raw shape:  (2251320, 4)
Days:  365


### Communtiy Level Aggregation

In [12]:
zones = pd.read_csv('/home/urwa/Documents/Projects/NYU Remote/project/UrbanTemporalNetworks/Data/ZonetoComm.csv')
zones.head(2)

,start_id,start_community
0,1,0.0
1,2,4.2


In [13]:
zones['start_community'] = zones.start_community.astype(int).astype(str)

zontoComm = dict(zip(zones.start_id.values,zones.start_community.values))

In [15]:
data['DOLocationID'] = data['DOLocationID'].apply(lambda x:zontoComm[x])

In [16]:
data.head(3)

,DOLocationID,Date,Hour,vehicle_count
0,0,2018-01-01,0,0.0
1,4,2018-01-01,0,0.0
2,2,2018-01-01,0,0.0


In [17]:
data = getTimeSeries(data)
data = data.reset_index()
data.head(3)

DOLocationID,Date,Hour,0,1,2,3,4,5
0,2018-01-01,0,297,39,18,57,16,0
1,2018-01-01,1,466,94,9,90,25,1
2,2018-01-01,2,417,97,23,101,40,0


### Merge External Data Features

In [18]:
externalDataDir = "/home/urwa/Documents/Projects/NYU Remote/project/data/HongData/"
extFile = externalDataDir + hub.upper() + ".csv"

In [19]:
extDf = pd.read_csv(extFile)
print(extDf.shape)
extDf.head(2)

(8760, 46)


,date,arrival,fhv,yellow,vehicle,ifmon,iftue,ifwed,ifthu,iffri,...,maxtemp,mintemp,avgtemp,departure,hdd,cdd,participation,newsnow,snowdepth,ifSnow
0,18/1/1 0:00,0,147,319,466,1,0,0,0,0,...,19,7,13.0,-20.4,52,0,0.0,0.0,0,0
1,18/1/1 1:00,1,347,397,744,1,0,0,0,0,...,19,7,13.0,-20.4,52,0,0.0,0.0,0,0


In [20]:
extDf['date'] = pd.to_datetime(extDf['date'], yearfirst=True)
extDf.head(2)

,date,arrival,fhv,yellow,vehicle,ifmon,iftue,ifwed,ifthu,iffri,...,maxtemp,mintemp,avgtemp,departure,hdd,cdd,participation,newsnow,snowdepth,ifSnow
0,2018-01-01 00:00:00,0,147,319,466,1,0,0,0,0,...,19,7,13.0,-20.4,52,0,0.0,0.0,0,0
1,2018-01-01 01:00:00,1,347,397,744,1,0,0,0,0,...,19,7,13.0,-20.4,52,0,0.0,0.0,0,0


In [21]:
min(extDf.date), max(extDf.date)

(Timestamp('2018-01-01 00:00:00'), Timestamp('2018-12-31 23:00:00'))

In [22]:
extDf['Hour'] = extDf['date'].dt.hour
extDf['Dow'] = extDf['date'].dt.dayofweek
extDf['Date'] = extDf['date'].dt.date

In [23]:
extDf.columns

Index(['date', 'arrival', 'fhv', 'yellow', 'vehicle', 'ifmon', 'iftue',
       'ifwed', 'ifthu', 'iffri', 'ifsat', 'ifsun', 'if0', 'if1', 'if2', 'if3',
       'if4', 'if5', 'if6', 'if7', 'if8', 'if9', 'if10', 'if11', 'if12',
       'if13', 'if14', 'if15', 'if16', 'if17', 'if18', 'if19', 'if20', 'if21',
       'if22', 'if23', 'maxtemp', 'mintemp', 'avgtemp', 'departure', 'hdd',
       'cdd', 'participation', 'newsnow', 'snowdepth', 'ifSnow', 'Hour', 'Dow',
       'Date'],
      dtype='object')

In [24]:
selected_columns = ['Date', 'Hour', 'Dow', 'arrival','maxtemp', 'mintemp', 'avgtemp', 'departure', 'hdd',
       'cdd', 'participation', 'newsnow', 'snowdepth', 'ifSnow']

In [25]:
extDf = extDf[selected_columns]

In [26]:
print(data.shape)
print(extDf.shape)

(8760, 8)
(8760, 14)


In [27]:
data['Date'] = pd.to_datetime(data['Date'])
extDf['Date'] = pd.to_datetime(extDf['Date'])

In [28]:
data = pd.merge(data,extDf, on=['Date', 'Hour'], how='inner')
print(data.shape)
data['Date'] = data['Date'].dt.date
data.head()

(8760, 20)


,Date,Hour,0,1,2,3,4,5,Dow,arrival,maxtemp,mintemp,avgtemp,departure,hdd,cdd,participation,newsnow,snowdepth,ifSnow
0,2018-01-01,0,297,39,18,57,16,0,0,0,19,7,13.0,-20.4,52,0,0.0,0.0,0,0
1,2018-01-01,1,466,94,9,90,25,1,0,1,19,7,13.0,-20.4,52,0,0.0,0.0,0,0
2,2018-01-01,2,417,97,23,101,40,0,0,1,19,7,13.0,-20.4,52,0,0.0,0.0,0,0
3,2018-01-01,3,297,74,22,78,35,0,0,0,19,7,13.0,-20.4,52,0,0.0,0.0,0,0
4,2018-01-01,4,175,41,21,39,35,0,0,0,19,7,13.0,-20.4,52,0,0.0,0.0,0,0


### Train Test split

In [29]:
sep = int(0.75*len(data))
sep

6570

In [30]:
trainData = data[:sep]
testData = data[sep:]

In [31]:
trainData.shape

(6570, 20)

In [32]:
testData.shape

(2190, 20)

### Lag Variables

In [33]:
trainData.columns

Index(['Date', 'Hour', '0', '1', '2', '3', '4', '5', 'Dow', 'arrival',
       'maxtemp', 'mintemp', 'avgtemp', 'departure', 'hdd', 'cdd',
       'participation', 'newsnow', 'snowdepth', 'ifSnow'],
      dtype='object')

In [34]:
lagColumns = ['0', '1', '2', '3', '4', '5', 'arrival']

DateColumns = ['Date']

targetColumns = ['0', '1', '2', '3', '4', '5']

In [35]:
maxlag = 12

dataset_train = addLag(trainData, maxlag, lagColumns)

dataset_train.shape

(6558, 104)

In [36]:
dataset_test = addLag(testData, maxlag, lagColumns)
dataset_test.shape

(2178, 104)

### Modelling

In [37]:
X_train = dataset_train.drop(targetColumns+DateColumns , axis = 1)
X_test = dataset_test.drop(targetColumns+DateColumns , axis = 1)
y_train = dataset_train[targetColumns]
y_test = dataset_test[targetColumns]

In [38]:
X_train.shape, X_test.shape

((6558, 97), (2178, 97))

In [39]:
y_train.shape, y_test.shape

((6558, 6), (2178, 6))

### Hyperparameter Tuning

In [40]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 150, stop = 300, num = 3)]
# Number of features to consider at every split
max_features = ['sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(50, 110, num = 5)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2,3,4]
# Minimum number of samples required at each leaf node
min_samples_leaf = [2,3]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [150, 225, 300], 'max_features': ['sqrt'], 'max_depth': [50, 65, 80, 95, 110, None], 'min_samples_split': [2, 3, 4], 'min_samples_leaf': [2, 3], 'bootstrap': [True, False]}


In [41]:
if tune_hyp_params:
    rf = RandomForestRegressor()
    rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 5, \
                                   cv = 5, verbose=2, random_state=42, n_jobs = -1)
    rf_random.fit(X_train, y_train)
    rf_random.best_params_

### Training the Best Model

In [42]:
rf2 = RandomForestRegressor(random_state = 2019, n_estimators=150, 
                           min_samples_split=3,
                           min_samples_leaf= 2, 
                           max_features= 'sqrt',
                           max_depth= None, 
                           bootstrap= False)

In [43]:
rf2.fit(X_train,y_train)

RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=None,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=2, min_samples_split=3,
           min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=None,
           oob_score=False, random_state=2019, verbose=0, warm_start=False)

In [44]:
rf2.score(X_train,y_train)

0.9971682075058597

In [45]:
rf2.score(X_test,y_test)

0.8928205222553571

### Predict

In [46]:
network_prediction = rf2.predict(X_test)
network_prediction.shape

(2178, 6)

In [47]:
y_test.shape

(2178, 6)

### Evaluate

In [48]:
get_rmse(y_test, network_prediction)

0    67.286696
1    13.110755
2     4.812409
3    20.286431
4     6.652493
5     0.695851
dtype: float64

In [49]:
r2_score(y_test, network_prediction, multioutput='variance_weighted')

0.8928205222553571

In [50]:
y_test.mean()

0    415.661157
1     31.051423
2     11.749770
3     80.798898
4     20.221304
5      0.378788
dtype: float64